In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_821959/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_47_3,0.999909,0.827148,0.817581,0.999810,0.000377,1.155861,0.652501,0.000890,0.042053,0.019426,1.005802,0.019732,145.764659,287.099832,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_47_4,0.999909,0.827218,0.817136,0.999803,0.000378,1.155394,0.654095,0.000924,0.042455,0.019431,1.005805,0.019737,145.763590,287.098762,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_47_1,0.999906,0.827358,0.818571,0.999860,0.000391,1.154457,0.648960,0.000659,0.043796,0.019776,1.006013,0.020088,145.693064,287.028236,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_47_2,0.999906,0.827380,0.818516,0.999856,0.000391,1.154308,0.649157,0.000678,0.044512,0.019783,1.006017,0.020095,145.691716,287.026888,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_47_5,0.999905,0.826914,0.816587,0.999766,0.000394,1.157429,0.656058,0.001099,0.037870,0.019847,1.006056,0.020159,145.678829,287.014002,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,model_1_2_4,0.934765,0.761141,0.776830,0.863829,0.271544,1.597251,0.798265,0.156640,1.171599,0.521099,5.175059,0.529305,132.607265,273.942437,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
475,model_1_2_3,0.904421,0.726570,0.773982,0.868213,0.397850,1.828429,0.808453,0.151597,1.158089,0.630753,7.117047,0.640687,131.843363,273.178535,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
476,model_1_2_2,0.899537,0.716011,0.778164,0.864262,0.418182,1.899038,0.793495,0.156142,1.079427,0.646670,7.429659,0.656854,131.743679,273.078851,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
477,model_1_2_1,0.897750,0.714188,0.777383,0.860902,0.425620,1.911228,0.796290,0.160007,1.088220,0.652396,7.544026,0.662670,131.708416,273.043589,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
